# Rule Engine DSL

A rule consists of a __Condition__ and an __Action__. A condition demarcates a situation where the concomitant action is to applied.

In [ ]:
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent"

Installed Packages Microsoft.Diagnostics.Tracing.TraceEvent, 2.0.74

In [ ]:
open Microsoft.Diagnostics.Tracing.Analysis.GC
open System

## Condition 

In [ ]:
// Format: EventName.Property Condition Value
// For example: GCEnd.AllocationRate LessThan 100
type ConditionerEvent = 
    | GCEnd

type ConditionerProperty = 
    | SuspensionTimeMSec
    | PauseTimeMSec

type Conditioner = { ConditionerEvent: ConditionerEvent; ConditionerProperty : ConditionerProperty }

type ConditionType = 
    | LessThan 
    | LessThanEqualTo
    | GreaterThan
    | GreaterThanEqualTo
    | Equal
    | NotEqual
    | IsAnomaly

type AnomalyDetectionMethod =
    | Max 

type ConditionalValue = 
    | Value                  of double 
    | AnomalyDetectionMethod of AnomalyDetectionMethod 

type Condition = 
    {  Conditioner      : Conditioner;
       ConditionType    : ConditionType;
       ConditionalValue : ConditionalValue }

## Action

In [ ]:
// For example: <Condition>;Print CallStack

type ActionOperator = 
    |  Print

type ActionOperand =
    | Alert
    | CallStack
    //| HeapStats

type Action = { ActionOperator: ActionOperator; ActionOperand: ActionOperand }

## Rule: Combining Conditions and Actions

In [ ]:
type Rule = { Condition : Condition; Action : Action; }

## Parser

The goal is to parse the following types of Rules:

1. ``GCEnd.PauseTimeMSec > 100 : Print CallStack``
2. ``GCEnd.SuspensionDurationMSec IsAnomaly Spike: Print HeapStat``
3. ``GCEnd.PauseDurationMSec >= 100: Print Alert``

### Parser Logic
1. Split on ``:``.
2. First element of the split is the Condition.
   1. Match the Condition Event and Property and associate them with a real type. 
   2. Match the Condition Type and the Value and match them with a real func.
3. Second element of the split is the Action.
   1. Match the Action Operator and the Operator.

### Parse Condition

In [ ]:
let parseCondition (conditionAsString : string) : Condition = 

    let splitCondition : string[] = conditionAsString.Split(" ", StringSplitOptions.RemoveEmptyEntries)
    
    // Precondition check
    if splitCondition.Length <> 3 
    then failwith("Incorrect format of the condition. Format is: Event.Property Condition ConditionalValue. For example: GCEnd.SuspensionTimeMSec >= 300")
    
    // Condition Event and Property
    let parseConditioner : Conditioner = 
        let splitConditioner : string[] = splitCondition.[0].Split(".", StringSplitOptions.RemoveEmptyEntries)
        let parseConditionEvent : ConditionerEvent = 
            match splitConditioner.[0].ToLower() with
            | "gcend" -> ConditionerEvent.GCEnd
            | _       -> failwith($"{splitConditioner.[0]} is an unrecognized Conditioner Event.")

        let parseConditionProperty : ConditionerProperty =
            match splitConditioner.[1].ToLower() with
            | "suspensiontimemsec" -> ConditionerProperty.SuspensionTimeMSec
            | "pausetimemsec"      -> ConditionerProperty.PauseTimeMSec 
            | _                    -> failwith($"{splitConditioner.[1]} is an unrecognized Conditioner Property")

        { ConditionerEvent = parseConditionEvent; ConditionerProperty = ConditionerProperty.PauseTimeMSec }

    // Condition Type
    let parseConditionType : ConditionType =
        match splitCondition.[1].ToLower() with
        | ">"  | "greaterthan"                                 -> ConditionType.GreaterThan 
        | "<"  | "lessthan"                                    -> ConditionType.LessThan
        | ">=" | "greaterthanequalto" | "greaterthanorequalto" -> ConditionType.GreaterThanEqualTo
        | "<=" | "lessthanequalto"    | "lessthanorequalto"    -> ConditionType.LessThanEqualTo
        | "="  | "equal"              | "equals"               -> ConditionType.Equal
        | "!=" | "notequal"                                    -> ConditionType.NotEqual
        | "isanomaly"                                          -> ConditionType.IsAnomaly
        | _                                                    -> failwith("${splitCondition.[1]} is an unrecognized condition type.")

    // Condition Value
    let parseConditionValue : ConditionalValue =
        let conditionalValueAsString = splitCondition.[2]
        let checkDouble, doubleValue = Double.TryParse conditionalValueAsString 
        if checkDouble then ConditionalValue.Value(doubleValue) 
        else match conditionalValueAsString.ToLower() with
        | "max" -> ConditionalValue.AnomalyDetectionMethod( AnomalyDetectionMethod.Max )
        | _     -> failwith($"{conditionalValueAsString} is an unrecognized Anomaly Detection Method.")

    { Conditioner = parseConditioner; ConditionType = parseConditionType; ConditionalValue = parseConditionValue }

### Testing Conditional Parsing

In [ ]:
let testConditional1   = "GCEnd.SuspensionTimeMSec > 100"
let parsedConditional1 = parseCondition testConditional1
display(parsedConditional1)

Conditioner,ConditionType,ConditionalValue
"{ { ConditionerEvent = GCEnd ConditionerProperty = PauseTimeMSec }: ConditionerEvent: GCEnd, ConditionerProperty: PauseTimeMSec }",GreaterThan,{ Value 100.0: Item: 100 }


In [ ]:
let testConditional2   = "GCEnd.SuspensionTimeMSec isAnomaly Max"
let parsedConditional2 = parseCondition testConditional2
display(parsedConditional2)

Conditioner,ConditionType,ConditionalValue
"{ { ConditionerEvent = GCEnd ConditionerProperty = PauseTimeMSec }: ConditionerEvent: GCEnd, ConditionerProperty: PauseTimeMSec }",IsAnomaly,{ AnomalyDetectionMethod Max: Item: Max }


### Parse Action

In [ ]:
let parseAction (actionAsAString : string) : Action = 
    let splitAction : string[] = actionAsAString.Split(" ", StringSplitOptions.RemoveEmptyEntries)

    // ActionOperator
    let parseActionOperator : ActionOperator = 
        match splitAction.[0].ToLower() with
        | "print" -> ActionOperator.Print
        | _       -> failwith($"{splitAction.[0]} is an unrecognized Action Operator.")

    // ActionOperand 
    let parseActionOperand : ActionOperand = 
        match splitAction.[1].ToLower() with
        | "alert"     -> ActionOperand.Alert
        | "callstack" -> ActionOperand.CallStack
        | _           -> failwith($"{splitAction.[1]} is an unrecognized Action Operand.")

    { ActionOperator = parseActionOperator; ActionOperand = parseActionOperand }

### Testing Action Parsing

#### Success Case

In [ ]:
let testActionParsing : string = "Print CallStack"
display(parseAction testActionParsing)

ActionOperator,ActionOperand
Print,CallStack


#### Failure Case

In [ ]:
let testActionParsingWithFailure : string = "Print Value"
display(parseAction testActionParsingWithFailure)

Error: System.Exception: Value is an unrecognized Action Operand.
   at FSI_0177.parseAction(String actionAsAString)
   at <StartupCode$FSI_0179>.$FSI_0179.main@()

### Parsing a Rule

In [ ]:
let parseRule (ruleAsString : string) : Rule = 
    let splitRuleAsAString : string[] = ruleAsString.Split(":")
    let condition : Condition = parseCondition splitRuleAsAString.[0]
    let action : Action = parseAction splitRuleAsAString.[1]
    { Condition = condition; Action = action }

### Testing Parsing a Rule

In [ ]:
let testRule1 = "GCEnd.SuspensionTimeMSec > 100 : Print CallStack"
let parsedTestRule1 = parseRule testRule1
display(parsedTestRule1)

Condition,Action
"{ { Conditioner = { ConditionerEvent = GCEnd ConditionerProperty = PauseTimeMSec } ConditionType = GreaterThan ConditionalValue = Value 100.0 }: Conditioner: { { ConditionerEvent = GCEnd ConditionerProperty = PauseTimeMSec }: ConditionerEvent: GCEnd, ConditionerProperty: PauseTimeMSec }, ConditionType: GreaterThan, ConditionalValue: { Value 100.0: Item: 100 } }","{ { ActionOperator = Print ActionOperand = CallStack }: ActionOperator: Print, ActionOperand: CallStack }"


In [ ]:
let testRule2 = "GCEnd.PauseTimeMSec >= 300 : Print Alert"
let parsedTestRule2 = parseRule testRule2
display(parsedTestRule2)

Condition,Action
"{ { Conditioner = { ConditionerEvent = GCEnd ConditionerProperty = PauseTimeMSec } ConditionType = GreaterThanEqualTo ConditionalValue = Value 300.0 }: Conditioner: { { ConditionerEvent = GCEnd ConditionerProperty = PauseTimeMSec }: ConditionerEvent: GCEnd, ConditionerProperty: PauseTimeMSec }, ConditionType: GreaterThanEqualTo, ConditionalValue: { Value 300.0: Item: 300 } }","{ { ActionOperator = Print ActionOperand = Alert }: ActionOperator: Print, ActionOperand: Alert }"
